droped col[0] and added Scoring order in first col

In [2]:
try:    
    import pandas as pd    
    from datetime import datetime    
    
    # Define maximum points for each category    
    fundamentals_points = 12    
    profitability_points = 16    
    upside_points = 7    
    growth_points = 10    
    momentum_points = 22    
    valuations_points = 8    
    
    # Load the Excel files with the correct header row    
    df1 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Fundamentals_202501151954.xlsx', header=1)    
    df2 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Profitability_202501152033.xlsx', header=1)    
    df3 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Upside_202501152045.xlsx', header=1)    
    df4 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Valuation_202501152034.xlsx', header=1)    
    df5 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Growth_202501152032.xlsx', header=1)    
    df6 = pd.read_excel(r'c:\Users\ext_luleb\Downloads\Companies in Momentum_202501152034.xlsx', header=1)    
    
    # Ensure the 'Company' column is properly named    
    df1 = df1.rename(columns={df1.columns[1]: 'Company'})    
    df2 = df2.rename(columns={df2.columns[1]: 'Company'})    
    df3 = df3.rename(columns={df3.columns[1]: 'Company'})    
    df4 = df4.rename(columns={df4.columns[1]: 'Company'})    
    df5 = df5.rename(columns={df5.columns[1]: 'Company'})    
    df6 = df6.rename(columns={df6.columns[1]: 'Company'})    
    
    # Merge the DataFrames on the "Company" column with custom suffixes    
    df_all = (    
        df1.merge(df2, on='Company', suffixes=('_df1', '_df2'))    
           .merge(df3, on='Company', suffixes=('', '_df3'))    
           .merge(df4, on='Company', suffixes=('', '_df4'))    
           .merge(df5, on='Company', suffixes=('', '_df5'))    
           .merge(df6, on='Company', suffixes=('', '_df6'))    
    )    
    
    # Drop columns that start with "Total"    
    df_all = df_all.loc[:, ~df_all.columns.str.startswith('Total')]    
    
    # Calculate the scores for all columns with specified prefixes    
    prefixes = {    
        'Fundamentals': fundamentals_points,    
        'Profitability': profitability_points,    
        'Upside': upside_points,    
        'Growth': growth_points,    
        'Momentum': momentum_points,    
        'Valuations': valuations_points    
    }    
    
    # Loop through each prefix and calculate the score    
    for prefix, points in prefixes.items():    
        score_column_name = f"{prefix}_score"    
        # Select columns that start with the given prefix    
        selected_columns = df_all.filter(regex=f'^{prefix}')    
        # Calculate the score for each row    
        df_all[score_column_name] = selected_columns.multiply(points, axis=1).sum(axis=1)    
    
    # Calculate the row-wise average and sum    
    df_all['Average'] = df_all.filter(regex='^(' + '|'.join(prefixes.keys()) + ')_score').mean(axis=1)    
    df_all['Sum'] = df_all.filter(regex='^(' + '|'.join(prefixes.keys()) + ')_score').sum(axis=1)    
    
    # Sort by the "Average" column in descending order    
    df_all = df_all.sort_values(by='Average', ascending=False)    
    
    # Drop the first column by index    
    df_all = df_all.drop(df_all.columns[0], axis=1)    
    
    # Add the "Scoring order" column with incremental values    
    df_all.insert(0, 'Scoring order', range(1, len(df_all) + 1))    
    
    # Display the first few rows of the modified DataFrame    
    print(df_all.head())    
    
    # Generate timestamp    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')    
    # Export the modified DataFrame to an Excel file    
    output_path = fr'c:\Users\ext_luleb\Downloads\Merged_Companies_Processed_{timestamp}.xlsx'    
    df_all.to_excel(output_path, index=False)    
    
except Exception as e:    
    print("An error occurred:", e)    

    Scoring order Company  Fundamentals  Debt To Assets (Quarterly)  \
39              1  DIG.WA            58                       24.91   
45              2  XTB.WA            57                        0.47   
8               3  SNT.WA            72                        4.89   
18              4  RBW.WA            67                        5.10   
79              5  CBF.WA            47                       30.59   

    Altman Z-Score (Ttm)  Market Cap (Usd)  #_df2  Profitability  \
39                  5.53      6.332108e+07      3             88   
45                   NaN      2.019171e+09      2            100   
8                   7.58      4.342186e+08     22             69   
18                  5.57      4.353772e+08     21             69   
79                  4.10      5.112276e+08     38             58   

    Return On Invested Capital (Annual)  \
39                                25.75   
45                                44.84   
8                                 3